In [13]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

df = pd.read_csv('data.csv', parse_dates=['Transaction Date'])
#Clear non needed types
df = df[~df['Type'].isin(['Municipal Security', 'Other Securities','Non-Public Stock', 'Corporate Bond'])]

In [14]:
#Drop exchanges
df = df[df['Buy/Sell'] != 'Exchange']

In [15]:
#irregular stocks to fix
irregular = df[df['Stock'].apply(lambda x: len(x) > 5)]
#drop all rows that are weird in the stock column;
irregular = irregular[~irregular['Stock'].str.contains('bond|bds|due|%', case=False)]

In [18]:
import json
#try to search up ticker
driver = webdriver.Firefox()
names = {}
#workaround for startup web scraping block - going slowly
count = 0
sleep = 0
for stock in irregular['Stock']:
    count += 1
    sleep += 1
    driver.get('https://www.startpage.com')
    search_bar = driver.find_element(By.ID, 'q') 
    og = stock
    if '\n' in stock:
        stock = stock.split('\n')[0]
    search_bar.send_keys(f"{stock} stock symbol yahoo finance")
    search_bar.send_keys(Keys.RETURN)
    time.sleep(6) 
    try: #yahoo
        search_results = driver.find_element(By.XPATH, "//a[contains(@href, 'https://finance.yahoo.com/quote/')]")
        search_results = search_results.text.split('/')[-2]
        names[og] = search_results
    except:
        names[og] = 'N/A'
    time.sleep(6)
    if count == 20:
        count = 0
        time.sleep(600)
    if sleep == 200:
        sleep = 0
        time.sleep(6000)
driver.quit()
with open('clean_irregulars.json', 'w') as f:
    json.dump(names, f)


,Stock,Transaction Date,Buy/Sell,Type,Amount,Senator
0,MDLZ (stock option)\nOption Type: Neither\nStr...,2015-12-30,Sale,Stock Option,"$50,001 - $100,000",Roy Blunt
1,MDLZ (stock option),2014-12-30,Sale,NaN,"$50,001 - $100,000",Roy Blunt
2,The Kraft Heinz Company - Common Stock\nOption...,2022-12-23,Sale,Stock Option,"$1,001 - $15,000",Roy Blunt
3,AMGEN INC (AMGN),2015-11-23,Purchase,Stock,"$1,001 - $15,000",Shelley m Capito
4,ETN-Eaton Corporation plc,2017-10-25,Sale,Stock,"$1,001 - $15,000",Shelley m Capito
...,...,...,...,...,...,...
1100,CHL CHINA MOBILE LTD,2016-09-22,Sale,Stock,"$1,001 - $15,000",Sheldon Whitehouse
1101,STT-State Street Corporation,2016-04-12,Sale,Stock,"$15,001 - $50,000",Sheldon Whitehouse
1102,COF-PP,2014-10-20,Sale,NaN,"$15,001 - $50,000",Sheldon Whitehouse
1103,JTKWY - Just Eat Takeaway.com NV - ADR,2022-11-11,Sale,Stock,"$1,001 - $15,000",Ron l Wyden


In [ ]:
with open('clean_irregulars.json', 'r') as f:
    loaded_data = json.load(f)
    print(loaded_data)

[('MDLZ', 'MDLZ (stock option)'), ('MDLZ', 'MDLZ (stock option)'), ('KHC', 'The Kraft Heinz Company - Common Stock'), ('AMGN', 'AMGEN INC (AMGN)'), ('ETN', 'ETN-Eaton Corporation plc'), ('SAFRY', 'SAFRY - Safran SA'), ('EADSY', 'EADSY - Airbus SE'), ('AAPL', 'AAPL-Apple Inc.'), ('AAGIY', 'AAGIY - AIA Group Limited'), ('HLT', 'HLT-Hilton Worldwide Holdings Inc.'), ('WEC', 'WEC-WEC Energy Group, Inc.'), ('IBM.MX', 'IBM.MX'), ('PG', 'PG-The Procter & Gamble Company'), ('PG', 'PG-The Procter & Gamble Company'), ('IBM.MX', 'IBM.MX'), ('WFC', 'Wells Fargo & Company'), ('AAPL', 'Apple Inc.'), ('UNH', 'UnitedHealth Group Incorporated Common Stock (DE)'), ('SIQ.DE', 'SIGMA ALDRICH CORP'), ('AMS.MC', 'Amadeus IT Group, S.A.')]
